In [36]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [37]:
train_df = pd.read_csv('./titanicTrain.csv')
test_df = pd.read_csv('./titanicQuestion.csv')

In [38]:
train_df = train_df[:1000]
print('Train:', len(train_df),
      'Test:', len(test_df))

Train: 1000 Test: 309


## preprocessing function

In [39]:
# Titanic data preprocessing function
def preprocessTitanicData(raw_df):
    df = raw_df[['survived', 'pclass', 'sibsp', 'parch', 'age', 'sex', 'fare', 'embarked']]
    df['age'] = df['age'].fillna(df['age'].median())
    df['fare'] = df['fare'].fillna(df['fare'].median())
    df['sex'] = df['sex'].map({'female':0, 'male':1}).astype(int)
    x_onehot_df = pd.get_dummies(data=df, columns= ['embarked'])
    ndarray = x_onehot_df.values
    feature = ndarray[:,1:]
    label = ndarray[:, 0]
    
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaledFeatures = minmax_scale.fit_transform(feature)
    
    return scaledFeatures, label

In [40]:
train_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [41]:
x_train, y_train = preprocessTitanicData(train_df)
x_test, y_test = preprocessTitanicData(test_df)

D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [42]:
print(x_train.shape, y_train.shape) #np.array np.array
print(x_test.shape, y_test.shape) # np.array np.array

# reshape y_train and y_test
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

print(x_train.shape, y_train.shape) #np.array np.array
print(x_test.shape, y_test.shape) # np.array np.array

(1000, 9) (1000,)
(309, 9) (309,)
(1000, 9) (1000, 1)
(309, 9) (309, 1)


## construct model

In [43]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms as transforms

In [44]:
DROPOUT_RATIO = 0.35

In [79]:
class LR(nn.Module): # 繼承torch.nn.module特性
    
    # 架構
    def __init__(self):
        super(LR, self).__init__() # super(LR, self) 繼承自己的__init__()
        self.hidden1 = nn.Linear(9, 100) # input 9, hidden 40  Linear全連接
        self.dropout = nn.Dropout(DROPOUT_RATIO)
        self.hidden2 = nn.Linear(100, 80) # input 40, hidden 40  Linear全連接
       # self.hidden22 = nn.Linear(100, 40)
        self.hidden3 = nn.Linear(80, 60) # input 40, hidden 40  Linear全連接
       # self.hidden3 = nn.Bilinear(40, 40, 30)
        self.hidden4 = nn.Linear(60, 40) # input 40, hidden 30  Linear全連接
        self.hidden5 = nn.Linear(10, 1) # hidden 10, output 1

    # 運作流程（資料流）
    def forward(self, x):  
        x = F.sigmoid(self.hidden1(x)) # 讓x由hidden1後，進入activation function
        x = F.dropout(self.dropout(x))
        
        #y = F.relu(self.hidden22(x))
        #y = F.dropout(self.dropout(y))
        
        x = F.relu(self.hidden2(x))
        x = F.dropout(self.dropout(x))
        
        x = F.relu(self.hidden3(x))
       # x = F.relu(self.hidden3(x, y))
        x = F.dropout(self.dropout(x))
        x = F.relu(self.hidden4(x))
        out = self.hidden5(x)
        return out
    

In [80]:
learning_rate = 0.03
model = LR()
loss_func = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #model.parameters() 所有訓練的參數都在這
# model.cuda()   

## Training

In [81]:
epochs = 5000

In [82]:
for epoch in range(epochs):
    epoch += 1

    # convert to variables
    x = Variable(torch.from_numpy(x_train).float())
    y = Variable(torch.from_numpy(y_train).float())

    # clear gradient w.r.t. parameters 
    optimizer.zero_grad()
    
    # forward to get output
    prediction = model(x)

    # calculate loss
    loss = loss_func(prediction, y)

    # backward to get gradient
    loss.backward()

    # update parameters
    optimizer.step() 

    if epoch % 10 == 0:
    # plot and show learning process
        print("epoch %d, loss %.8f" % (epoch, loss.data[0]))

RuntimeError: size mismatch, m1: [1000 x 40], m2: [10 x 1] at c:\users\administrator\downloads\new-builder\win-wheel\pytorch\aten\src\th\generic/THTensorMath.c:2033

In [ ]:
# get filters from parameters:
print(list(model.parameters())[0].size())  

In [ ]:
train_prediction = prediction.data.numpy()
train_prediction = np.where(train_prediction>=0.7,1,0)
train_df['prediction'] = train_prediction

## Prediction

In [ ]:
x_test_v = Variable(torch.from_numpy(x_test).float())
test_prediction = model(x_test_v).data.numpy()
# test_prediction = np.where(test_prediction>=0.6,1,0)

In [ ]:
test_df['prediction'] = test_prediction

In [ ]:
def apply_sub(row, label):
    if row[label] >= 0.5:
        return 1
    else:
        return 0

test_df['prediction'] = test_df.apply(lambda x: apply_sub(x, 'prediction'), axis = 1)
test_df['prediction']

In [ ]:
#test_df['prediction'].to_csv('titanic_answer_trained.csv')

## Verify model

In [ ]:
answer = pd.read_csv('titanicAns.csv')
accuracy = 1.0 - sum(abs(answer['survived'] - test_df['prediction'])) / test_df.shape[0]
print('Accuracy: %f %%'%(accuracy*100))

### dump accuracy

In [ ]:
with open('log.txt', 'a') as flog:
    flog.write('RESULT:\n\tDropout:\t%.3f\n\tepochs:\t\t%ilearning_rate:\t%f\n\tAccuracy:\t%f %%\n'%(DROPOUT_RATIO, epochs, learning_rate, accuracy*100))

In [ ]:
with open('log.txt', 'r') as flog:
    print(flog.read())